For Hitter data set, do different subset selection methods
- Best Subset selection (not solved...taking too much time without giving output)
- Forward Stepwise selection
- Backward Stepwise selection

In [198]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS,SequentialFeatureSelector as SFS
import statsmodels.api as sm

Load and clean data

In [206]:
#data=pd.read_csv(os.path.abspath("Hitters.csv"))
data=sm.datasets.get_rdataset("Hitters", package="ISLR").data.dropna()

data.loc[data['League'] == 'A', 'League'] = 1
data.loc[data['League'] == 'N', 'League'] = 0
data.loc[data['Division'] == 'W', 'Division'] = 1
data.loc[data['Division'] == 'E', 'Division'] = 0
data.loc[data['NewLeague'] == 'A', 'NewLeague'] = 1
data.loc[data['NewLeague'] == 'N', 'NewLeague'] = 0

data.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
rownames,,,,,,,,,,,,,,,,,,,,
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,0,1,632,43,10,475.0,0
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,1,1,880,82,14,480.0,1
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,0,0,200,11,3,500.0,0
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,0,0,805,40,4,91.5,0
-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,1,1,282,421,25,750.0,1


Data spliting

In [ ]:
train,test=train_test_split(data, test_size=0.2, random_state=42)
x_train = train.drop(columns='Salary')
x_test = test.drop(columns='Salary')
y_train = train['Salary']
y_test = test['Salary']

Generate subsets and fit with the data

In [138]:
def subset_selection(method,x,y):
    if method == 'exhaustive':
        selector = EFS(LinearRegression(),
                   min_features=1,
                   max_features=6,
                   scoring='r2',
                   cv=5,n_jobs=-1,print_progress=True)
    elif method == 'forward':
        selector = SFS(LinearRegression(),
                       k_features='best',forward=True,cv=5,
                       scoring='r2')
    elif method == 'backward':
        selector = SFS(LinearRegression(),
                       k_features='best',forward=False,cv=5,
                       scoring='r2')
    return selector.fit(x.values, y.values).subsets_ 

Select the best subset and return the feature names

In [139]:
def feature(method):
   subsets=subset_selection(method,x_train,y_train)
   subsets=pd.DataFrame(subsets).T
   model_id = subsets['avg_score'].idxmax().item()
   feature_names= x_train.columns[list(subsets.loc[model_id,'feature_idx'])]
   return feature_names

Fit the best subset and return the necessary results

In [140]:
models={}
def func(method, label, x_train, y_train, x_test, y_test):
   x_train_subset = x_train[feature(method)]
   x_test_subset = x_test[feature(method)]

   model = LinearRegression().fit(x_train_subset, y_train)
   models[label] = model
   
   # Calculate MSE metrics
   train_mse=np.mean((y_train - model.predict(x_train_subset))**2)
   test_mse=np.mean((y_test - model.predict(x_test_subset))**2)

    # Calculate R-squared metrics
   train_r2 = model.score(x_train_subset, y_train)
   test_r2 = model.score(x_test_subset, y_test)

   return {'train_mse':train_mse,'test_mse':test_mse,\
            'train_r2':train_r2,'test_r2':test_r2,\
            'intercept':np.round(model.intercept_,3),\
            'coefficients':np.round(model.coef_,3),\
            'model_number':len(feature(method)),\
            'feature_names':feature(method)}

Get output for all the methods

In [141]:
result_dict = {}
naming = [("forward", "Forward"), ("backward", "Backward")]
for method, label in naming:
    result_dict[method] = func(method, label, x_train, y_train, x_test, y_test)
result = pd.DataFrame(result_dict)
result

,forward,backward
train_mse,91151.280425,91151.280425
test_mse,136228.105852,136228.105852
train_r2,0.561677,0.561677
test_r2,0.246826,0.246826
intercept,29.969,29.969
coefficients,"[-1.565, 7.52, 2.945, 0.657, -32.288, -119.37,...","[-1.565, 7.52, 2.945, 0.657, -32.288, -119.37,..."
model_number,7,7
feature_names,"Index(['AtBat', 'Hits', 'Walks', 'CRBI', 'Leag...","Index(['AtBat', 'Hits', 'Walks', 'CRBI', 'Leag..."


For Hitter data set, do different shrinkage methods  
- Ridge 
- Lasso 
- Elastic Net


In [207]:
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

Load ISLR College dataset

In [ ]:
college = sm.datasets.get_rdataset("College", package="ISLR").data.dropna()
college['Private'] = (college['Private'] == 'Yes').astype(int)
college.head()

# Response and predictors
y = college['Apps'].values
x = college.drop(columns='Apps')
x_scaled=StandardScaler().fit_transform(x)

In [233]:
models={}
def output_function(x, y, alpha):
   np.random.seed(123)
   indices = np.arange(len(y))
   test_idx = np.random.choice(indices, size=350, replace=False)
   train_idx = np.setdiff1d(indices, test_idx)
   # Split the data into training and testing sets
   x_train, x_test = x[train_idx], x[test_idx]
   y_train, y_test = y[train_idx], y[test_idx]

   if alpha == 0:
      model = RidgeCV(alphas=np.logspace(-6, 6, 100), store_cv_results=True)
   elif alpha == 1:
      model = LassoCV(alphas=np.logspace(-6, 6, 100), cv=5, random_state=12, max_iter=10000)
   else:
      model = ElasticNetCV(alphas=np.logspace(-6, 6, 100), l1_ratio=alpha, cv=5, random_state=12, max_iter=10000)

   model.fit(x_train, y_train)
   best_lambda = np.round(model.alpha_,6).item()
   y_pred = model.predict(x_test)
   test_error = np.mean((y_test - y_pred) ** 2).item()

   models[str(alpha)] = np.round(model.coef_, 3)

   return {
        'best_lambda': best_lambda,
        'test_error': test_error
    }

In [234]:
output_function(x_scaled, y, alpha=0)    # Ridge
output_function(x_scaled, y, alpha=1)    # Lasso
output_function(x_scaled, y, alpha=0.5)  # Elastic Net

{'best_lambda': 1e-06, 'test_error': 1378634.9450623097}

In [235]:
print(models['0.5'])

[-1.598220e+02  4.399493e+03 -1.773177e+03  9.212110e+02 -3.237980e+02
  8.372110e+02  6.859600e+01 -3.010720e+02  1.049800e+02  4.798800e+01
 -7.131000e+00 -1.780900e+02  1.179000e+00  6.167600e+01  2.515400e+01
  3.226970e+02  1.022400e+02]
